In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("combined_data.csv", index_col=[0, 1])
df

Inflation  Population           GDP      Export  \
Country Code Year                                                     
ABW          1980         NaN     59909.0           NaN         NaN   
             1981         NaN     60563.0           NaN         NaN   
             1982         NaN     61276.0           NaN         NaN   
             1983         NaN     62228.0           NaN         NaN   
             1984         NaN     62901.0           NaN         NaN   
...                       ...         ...           ...         ...   
ZWE          2017    0.893962  14812482.0  2.096487e+10  107.151887   
             2018   10.618866  15034452.0  2.201518e+10  124.909506   
             2019  255.304991  15271368.0  2.062108e+10  131.425343   
             2020  557.201817  15526888.0  1.900914e+10  135.325610   
             2021   98.546105  15797210.0  2.061884e+10  185.806800   

                       Import  
Country Code Year              
ABW          1980         NaN  
             1981         NaN  
             1982         NaN  
             1983         NaN  
             1984         NaN  
...                       ...  
ZWE          2017   83.660837  
             2018  105.579357  
             2019   79.585840  
             2020   82.633621  
             2021  118.756526  

[7686 rows x 5 columns]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7686 entries, ('ABW', 1980) to ('ZWE', 2021)
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Inflation   6524 non-null   float64
 1   Population  7676 non-null   float64
 2   GDP         7282 non-null   float64
 3   Export      6718 non-null   float64
 4   Import      6718 non-null   float64
dtypes: float64(5)
memory usage: 329.7+ KB


In [4]:
df = df.dropna(subset = ['GDP'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7282 entries, ('ABW', 1986) to ('ZWE', 2021)
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Inflation   6474 non-null   float64
 1   Population  7282 non-null   float64
 2   GDP         7282 non-null   float64
 3   Export      6588 non-null   float64
 4   Import      6588 non-null   float64
dtypes: float64(5)
memory usage: 312.8+ KB


In [6]:
df = df.drop(index = [("AFG", 2020), ("AFG", 2021)])

In [7]:
df.isnull().sum()

Inflation     806
Population      0
GDP             0
Export        694
Import        694
dtype: int64

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 📌 بارگذاری داده‌ها (مثال فرضی)

# 📌 اجرای الگوریتم برای جایگذاری مقدارهای گمشده
for country in df.index.get_level_values("Country Code").unique():  # پردازش جداگانه برای هر کشور
    country_df = df.xs(country, level="Country Code")  # انتخاب داده‌های این کشور

    for column in df.columns:  # پردازش هر ستون عددی
        valid_data = country_df.dropna(subset=[column])  # داده‌های معتبر
        missing_data = country_df[country_df[column].isna()]  # داده‌های گمشده

        if missing_data.empty:
            continue  # اگر مقدار گمشده نداریم، رد شو

        X_valid = valid_data.index.values.reshape(-1, 1)  # سال‌های دارای مقدار
        y_valid = valid_data[column].values  # مقدار ویژگی سال‌های دارای داده
        X_missing = missing_data.index.values.reshape(-1, 1)  # سال‌های گمشده

        # 📌 افزودن مقدار ثابت برای مدل WLS
        X_valid_wls = sm.add_constant(X_valid)

        # 📌 محاسبه وزن‌ها برای WLS
        weights = 1 / (np.abs(X_valid - X_missing.T) + 1) ** 2  # وزن‌ها را محاسبه کن

        # 📌 اجرای مدل WLS
        model = sm.WLS(y_valid, X_valid_wls, weights.mean(axis=1))
        results = model.fit()

        # 📌 تنظیم صحیح ابعاد X_missing_wls
        X_missing_wls = np.ones((X_missing.shape[0], X_valid_wls.shape[1]))  # ایجاد ماتریس مناسب
        X_missing_wls[:, -1] = X_missing.ravel()  # تنظیم ستون دوم با سال‌ها

        # 📌 پیش‌بینی مقدارهای گمشده
        predicted_values = results.predict(X_missing_wls)

        # 📌 جایگذاری مقدارهای پیش‌بینی شده در DataFrame
        df.loc[(country, missing_data.index), column] = predicted_values


In [10]:
df

Inflation  Population           GDP      Export  \
Country Code Year                                                     
ABW          1986    1.073966     59931.0  1.040068e+09  105.022708   
             1987    3.643045     59159.0  1.207294e+09  115.503652   
             1988    3.121868     59331.0  1.432438e+09  135.066427   
             1989    3.991628     60443.0  1.606191e+09  101.349267   
             1990    5.836688     62753.0  1.669819e+09  123.596991   
...                       ...         ...           ...         ...   
ZWE          2017    0.893962  14812482.0  2.096487e+10  107.151887   
             2018   10.618866  15034452.0  2.201518e+10  124.909506   
             2019  255.304991  15271368.0  2.062108e+10  131.425343   
             2020  557.201817  15526888.0  1.900914e+10  135.325610   
             2021   98.546105  15797210.0  2.061884e+10  185.806800   

                       Import  
Country Code Year              
ABW          1986   47.432669  
             1987   58.364675  
             1988   83.150974  
             1989   95.561922  
             1990  132.571551  
...                       ...  
ZWE          2017   83.660837  
             2018  105.579357  
             2019   79.585840  
             2020   82.633621  
             2021  118.756526  

[7280 rows x 5 columns]

In [11]:
df.isnull().sum()

Inflation     0
Population    0
GDP           0
Export        0
Import        0
dtype: int64